# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
! pip install -q "datasets==2.15.0"
! pip install peft

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


In [18]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, AutoPeftModelForCausalLM, PeftModel

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# First experiment is we freeze all the model parameters.
for param in model.parameters():
    param.requires_grad = False

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [7]:
def text2tokens(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples['text'], return_tensors="pt", padding=True, truncation=True)


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(text2tokens, batched=True)


# Check that we tokenized the examples properly
assert tokenized_ds["train"][0]["input_ids"][:5] == [101, 2045, 2003, 2053, 7189]

# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[101, 2045, 2003, 2053, 7189, 2012, 2035, 2090, 3481, 3771, 1998, 6337, 2099, 2021, 1996, 2755, 2008, 2119, 2024, 2610, 2186, 2055, 6355, 6997, 1012, 6337, 2099, 3504, 15594, 2100, 1010, 3481, 3771, 3504, 4438, 1012, 6337, 2099, 14811, 2024, 3243, 3722, 1012, 3481, 3771, 1005, 1055, 5436, 2024, 2521, 2062, 8552, 1012, 1012, 1012, 3481, 3771, 3504, 2062, 2066, 3539, 8343, 1010, 2065, 2057, 2031, 2000, 3962, 12319, 1012, 1012, 1012, 1996, 2364, 2839, 2003, 5410, 1998, 6881, 2080, 1010, 2021, 2031, 1000, 17936, 6767, 7054, 3401, 1000, 1012, 2111, 2066, 2000, 12826, 1010, 2000, 3648, 1010, 2000, 16157, 1012, 2129, 2055, 2074, 9107, 1029, 6057, 2518, 2205, 1010, 2111, 3015, 3481, 3771, 3504, 2137, 2021, 1010, 2006, 1996, 2060, 2192, 1010, 9177, 2027, 9544, 2137, 2186, 1006, 999, 999, 999, 1007, 1012, 2672, 2009, 1005, 1055, 1996, 2653, 1010, 2030, 1996, 4382, 1010, 2021, 1045, 2228, 2023, 2186, 2003, 2062, 2394, 2084, 2137, 1012, 2011, 1996, 2126, 1010, 1996, 5889, 2024, 2428, 2204, 1998, 6

In [8]:
# since we are not training, and we need to use the pre-trained model as the baseline, we can directly 
# evaluate the model performance
# this is also because there are no trainable parameters for the model to train
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

count_trainable_parameters(model)

0

In [9]:
# metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Instantiate Trainer for evaluation
trainer = Trainer(
    model=model,
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer
    )
)

In [10]:
# Evaluate the model
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6902801394462585,
 'eval_accuracy': 0.508,
 'eval_runtime': 8.0771,
 'eval_samples_per_second': 61.903,
 'eval_steps_per_second': 7.8}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
# Configure LoRA
lora_config = LoraConfig(
    r=8, # Rank Number
    lora_alpha=32, # Alpha (Scaling Factor)
    lora_dropout=0.05, # Dropout Prob for Lora
    target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)

lora_model = get_peft_model(model, lora_config)

In [12]:
lora_model.print_trainable_parameters()

trainable params: 1,405,444 || all params: 67,768,324 || trainable%: 2.073895172617815


In [13]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer
    ),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.346628,0.858000
2,No log,0.365031,0.884000
3,No log,0.422678,0.864000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-32 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-64 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-96 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=96, training_loss=0.29294733206431073, metrics={'train_runtime': 79.5221, 'train_samples_per_second': 18.863, 'train_steps_per_second': 1.207, 'total_flos': 202448848896000.0, 'train_loss': 0.29294733206431073, 'epoch': 3.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.3466282784938812,
 'eval_accuracy': 0.858,
 'eval_runtime': 8.9264,
 'eval_samples_per_second': 56.014,
 'eval_steps_per_second': 0.896,
 'epoch': 3.0}

In [15]:
output_dir = "./lora_distilbert_imdb"

# Save the LoRA model and tokenizer, this is usually optional
lora_model.save_pretrained(output_dir)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
# Note that we already evaluated each model independently above, but to put both results side to side, we can load 
# the pre-trained lora weights, and compare its performance to the original model instance

In [19]:
# Load the LoRA adapter weights
lora_model = PeftModel.from_pretrained(model, output_dir)

In [22]:
# Instantiate Trainer for evaluation
trainer = Trainer(
    model=lora_model,
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer
    )
)
trainer.evaluate()

{'eval_loss': 0.34662824869155884,
 'eval_accuracy': 0.858,
 'eval_runtime': 8.5714,
 'eval_samples_per_second': 58.333,
 'eval_steps_per_second': 7.35}

We can see that we obtained the same evaluation accuracy of the peft model `0.858`, and comparing it to the random model result that was `0.508`, we can conclude that the peft technique improved the result.